In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import io, filters, exposure, morphology, measure, segmentation, img_as_float, img_as_uint
from scipy import ndimage
import glob
import tifffile
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from skimage.feature import peak_local_max

# ======= CONFIGURATION =======
# Set paths specifically for your Google Colab environment
INPUT_DIR = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised_ordered/Static-x40'
OUTPUT_DIR = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x40'

# Default EDF parameters
DEFAULT_MAX_Z_DIFF = 1      # Maximum allowed z-difference between adjacent pixels
DEFAULT_SIGMA = 1.0         # Sigma for Laplacian operator smoothing
DEFAULT_GAUSS_DENOISE = 0.5 # Sigma for Gaussian denoising

# Define channel names (customize these based on your data)
CHANNEL_NAMES = ['Cadherins', 'Nuclei', 'Golgi']

# ======= UTILITY FUNCTIONS =======

def create_dirs():
    """Create necessary directories for the new output structure"""
    os.makedirs(INPUT_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Create the new directory structure
    for channel in ['Cadherins', 'Nuclei', 'Golgi']:
        if channel == 'Nuclei':
            # Nuclei only has tophat
            os.makedirs(os.path.join(OUTPUT_DIR, channel, 'tophat'), exist_ok=True)
        else:
            # Cadherins and Golgi have both tophat and background
            os.makedirs(os.path.join(OUTPUT_DIR, channel, 'tophat'), exist_ok=True)
            os.makedirs(os.path.join(OUTPUT_DIR, channel, 'background'), exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# ======= BACKGROUND REMOVAL =======

def tophat_filter_background(image, radius=15):
    """Remove background using white top-hat filter"""
    # Normalize input to 0-1 range
    img = normalize_image(img_as_float(image))

    # Create structuring element (disk)
    selem = morphology.disk(radius)

    # Apply white top-hat filter (removes background while preserving foreground)
    tophat = morphology.white_tophat(img, selem)

    # Enhance contrast and ensure 0-1 range
    tophat = exposure.rescale_intensity(tophat)

    # Double-check normalization
    tophat = normalize_image(tophat)

    return tophat

# ======= REGIONAL GUIDED EDF METHOD =======

def edf_regional_guided(image_stack, region_size=15, overlap=5, sigma=1.0, gauss_denoise=0.5, max_z_diff=1):
    """
    Regional guided EDF with spatial consistency
    Processes image in overlapping regions to better handle local depth variations
    """
    # Convert to float and ensure 0-1 range
    stack = img_as_float(image_stack)
    stack = normalize_image(stack)
    z_size, height, width = stack.shape

    # Calculate focus measures
    focus_measures = np.zeros((z_size, height, width))

    for z in range(z_size):
        # Apply Gaussian filter to reduce noise
        if gauss_denoise > 0:
            blurred = filters.gaussian(stack[z], sigma=gauss_denoise)
        else:
            blurred = stack[z]

        # Apply Laplacian filter (detects edges/details)
        laplacian = np.abs(filters.laplace(blurred, ksize=3))

        # Apply small Gaussian to make the decision more robust
        if sigma > 0:
            focus_measures[z] = filters.gaussian(laplacian, sigma=sigma)
        else:
            focus_measures[z] = laplacian

    # Create output arrays
    # Changed from int32 to float64 to avoid casting error
    best_z = np.zeros((height, width), dtype=np.float64)
    weights = np.zeros((height, width), dtype=np.float32)

    # Process image in overlapping regions
    for y_start in range(0, height, region_size - overlap):
        for x_start in range(0, width, region_size - overlap):
            # Define region bounds
            y_end = min(y_start + region_size, height)
            x_end = min(x_start + region_size, width)

            # Extract region
            region_focus = focus_measures[:, y_start:y_end, x_start:x_end]

            # Process region with spatial continuity
            region_best_z = edf_with_spatial_continuity_region(
                region_focus,
                max_z_diff=max_z_diff
            )

            # Create weight mask (higher weights in center, lower at edges)
            y_grid, x_grid = np.mgrid[y_start:y_end, x_start:x_end]
            y_center = (y_start + y_end) / 2
            x_center = (x_start + x_end) / 2

            # Calculate distance from center (normalized to 0-1)
            y_dist = np.abs(y_grid - y_center) / (region_size / 2)
            x_dist = np.abs(x_grid - x_center) / (region_size / 2)
            dist = np.maximum(y_dist, x_dist)
            region_weights = np.clip(1.0 - dist, 0.1, 1.0)

            # Convert region_best_z to float64 before multiplying
            region_best_z_float = region_best_z.astype(np.float64)

            # Add weighted contribution to output
            best_z[y_start:y_end, x_start:x_end] += region_best_z_float * region_weights
            weights[y_start:y_end, x_start:x_end] += region_weights

    # Normalize by weights and round to nearest integer
    best_z = np.round(best_z / np.maximum(weights, 1e-6)).astype(np.int32)

    # Ensure best_z values are within valid range
    best_z = np.clip(best_z, 0, z_size - 1)

    # Create output by taking pixels from best z-slices
    result = np.zeros((height, width), dtype=np.float32)
    for z in range(z_size):
        mask = best_z == z
        result[mask] = stack[z][mask]

    # Create a visualization of the focus map (normalized to 0-1)
    focus_map = best_z / (z_size - 1)

    return result, focus_map, best_z

def edf_with_spatial_continuity_region(focus_measures, max_z_diff=1):
    """
    Helper function for regional guided EDF
    Applies spatial continuity to a region's focus measures
    """
    z_size, height, width = focus_measures.shape

    # Find initial z-slice with maximum focus for each pixel
    initial_best_z = np.argmax(focus_measures, axis=0)

    # Create a processed mask to track pixels that have been assigned a final z-value
    processed = np.zeros((height, width), dtype=bool)

    # Create the output z-map that will be filled with spatially consistent z-values
    final_best_z = np.copy(initial_best_z)

    # Function to get valid 4-connected neighbors
    def get_neighbors(y, x):
        neighbors = []
        for ny, nx in [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]:
            if 0 <= ny < height and 0 <= nx < width:
                neighbors.append((ny, nx))
        return neighbors

    # Find seed points - we'll start from pixels with highest confidence
    confidence = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            z = initial_best_z[y, x]
            confidence[y, x] = focus_measures[z, y, x]

    # Sort pixels by confidence (highest first)
    confidence_threshold = np.percentile(confidence, 95)  # Use top 5% as seeds
    seed_indices = np.where(confidence > confidence_threshold)
    seed_points = list(zip(seed_indices[0], seed_indices[1]))

    # Sort seed points by confidence (highest first)
    seed_points.sort(key=lambda idx: confidence[idx[0], idx[1]], reverse=True)

    # Initialize frontier with seed points
    frontier = seed_points.copy()
    for y, x in frontier:
        processed[y, x] = True

    # Process each point and its neighbors using a breadth-first approach
    while frontier:
        y, x = frontier.pop(0)
        current_z = final_best_z[y, x]

        # Check neighbors
        for ny, nx in get_neighbors(y, x):
            if processed[ny, nx]:
                continue

            # Find best z within allowed range from current pixel
            z_min = max(0, current_z - max_z_diff)
            z_max = min(z_size - 1, current_z + max_z_diff)

            # Extract the relevant slice of focus measures and find best z
            z_slice = focus_measures[z_min:z_max+1, ny, nx]
            relative_best_z = np.argmax(z_slice)
            final_best_z[ny, nx] = z_min + relative_best_z

            processed[ny, nx] = True
            frontier.append((ny, nx))

    # Check if there are any unprocessed pixels left (should be rare)
    unprocessed = ~processed
    if np.any(unprocessed):
        # Assign them the value of nearest processed neighbor
        dist, indices = ndimage.distance_transform_edt(
            unprocessed, return_indices=True)

        # Assign z-values from nearest processed pixel
        for y, x in zip(*np.where(unprocessed)):
            idx_y, idx_x = indices[0, y, x], indices[1, y, x]
            final_best_z[y, x] = final_best_z[idx_y, idx_x]

    return final_best_z

# ======= MAIN PROCESSING FUNCTIONS =======

def process_single_stack(stack, apply_tophat=False, max_z_diff=DEFAULT_MAX_Z_DIFF):
    """
    Process a single z-stack with the regional EDF method
    """
    print(f"Processing stack with shape {stack.shape}...")

    # Convert to float and ensure valid range (0-1)
    stack = img_as_float(stack)

    # Ensure stack values are in 0-1 range
    min_val = np.min(stack)
    max_val = np.max(stack)
    if min_val < 0 or max_val > 1:
        print(f"Normalizing stack from range [{min_val}, {max_val}] to [0, 1]")
        stack = normalize_image(stack)

    # Apply background removal if requested
    if apply_tophat:
        print("Applying tophat background removal...")
        processed_stack = np.zeros_like(stack, dtype=np.float32)
        for z in range(stack.shape[0]):
            processed_stack[z] = tophat_filter_background(stack[z])
        working_stack = processed_stack
    else:
        working_stack = stack

    # Store original slices for comparison
    mid_z_idx = stack.shape[0] // 2

    # Prepare results dictionary
    results = {
        'original_middle': stack[mid_z_idx],
        'original_max': np.max(stack, axis=0)
    }

    # Apply regional method
    print("Applying regional guided EDF...")
    proj, focus_map, _ = edf_regional_guided(
        working_stack, max_z_diff=max_z_diff)
    results['regional_edf'] = proj
    results['focus_map_regional'] = focus_map

    return results

def save_results_to_disk(results, filename, channel_name, apply_tophat):
    """Save results to disk with updated directory structure and naming"""
    base_name = os.path.splitext(filename)[0]

    # Get only the projection result (not focus map)
    key = "regional_edf"
    if key in results:
        # Determine processing type folder
        process_type = "tophat" if apply_tophat else "background"

        # Skip if this is not one of the required outputs
        if channel_name == "Nuclei" and not apply_tophat:
            print(f"Skipping {channel_name} without tophat (not required)")
            return

        # Create appropriate directory path
        output_dir = os.path.join(OUTPUT_DIR, channel_name, process_type)
        os.makedirs(output_dir, exist_ok=True)

        # Create output filename - using just the base name as requested
        output_filename = f"{base_name}_{channel_name}_regional"
        if apply_tophat:
            output_filename += "_tophat"
        output_filename += ".tif"

        output_path = os.path.join(output_dir, output_filename)

        # Normalize to 0-1 range and convert to uint16
        projection = results[key]
        projection_normalized = normalize_image(projection)
        projection_uint = img_as_uint(projection_normalized)

        # Save
        tifffile.imwrite(output_path, projection_uint)
        print(f"Saved {output_path}")

def process_directory(input_dir=INPUT_DIR, max_z_diff=DEFAULT_MAX_Z_DIFF,
                     file_pattern='*.tif*', save_results=True):
    """
    Process all files in a directory with channel-specific settings
    """
    # Get all matching files
    all_files = glob.glob(os.path.join(input_dir, file_pattern), recursive=True)
    all_files.sort()

    if not all_files:
        print(f"No files matching '{file_pattern}' found in {input_dir}")
        return []

    print(f"Found {len(all_files)} files to process")

    # Create output directories
    create_dirs()

    # Process each file
    processed_files = []

    for file_idx, file_path in enumerate(all_files):
        print(f"\nProcessing file {file_idx+1}/{len(all_files)}: {os.path.basename(file_path)}")

        try:
            results = process_file(
                file_path, max_z_diff=max_z_diff,
                save_results=save_results
            )

            if results:
                processed_files.append(file_path)
                print(f"Successfully processed {os.path.basename(file_path)}")

        except Exception as e:
            print(f"Error processing file {os.path.basename(file_path)}: {e}")
            import traceback
            traceback.print_exc()

    print(f"\nSuccessfully processed {len(processed_files)}/{len(all_files)} files")
    return processed_files

def process_file(file_path, max_z_diff=DEFAULT_MAX_Z_DIFF, save_results=True):
    """
    Process a single file with channel-specific processing:
    - Nuclei (channel 1): Apply tophat only
    - Cadherins and Golgi (channels 0 and 2): Create with and without tophat
    """
    filename = os.path.basename(file_path)
    print(f"Processing file: {filename}")

    # Load image
    try:
        image = tifffile.imread(file_path)
        print(f"Image shape: {image.shape}, dtype: {image.dtype}")
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

    # Handle different dimensionalities
    if len(image.shape) == 3:
        # Single channel z-stack - process with and without tophat
        results_no_tophat = process_single_stack(
            image, apply_tophat=False, max_z_diff=max_z_diff)

        results_tophat = process_single_stack(
            image, apply_tophat=True, max_z_diff=max_z_diff)

        if save_results:
            save_results_to_disk(results_no_tophat, filename, "default", False)
            save_results_to_disk(results_tophat, filename, "default", True)

        return {"default_no_tophat": results_no_tophat, "default_tophat": results_tophat}

    elif len(image.shape) == 4:
        # Multi-channel z-stack with channel-specific processing
        all_results = {}

        # Process each channel with appropriate settings
        for ch_idx in range(image.shape[0]):
            # Extract channel
            channel_data = image[ch_idx]
            ch_name = CHANNEL_NAMES[ch_idx] if ch_idx < len(CHANNEL_NAMES) else f"channel_{ch_idx}"

            # Apply channel-specific processing
            if ch_idx == 1:  # Nuclei (second channel) - tophat only
                print(f"Processing {ch_name} with tophat...")
                results = process_single_stack(
                    channel_data, apply_tophat=True, max_z_diff=max_z_diff)

                all_results[f"{ch_name}_tophat"] = results

                if save_results:
                    save_results_to_disk(results, filename, ch_name, True)

            else:  # Cadherins and Golgi (channels 0 and 2) - with and without tophat
                print(f"Processing {ch_name} without tophat...")
                results_no_tophat = process_single_stack(
                    channel_data, apply_tophat=False, max_z_diff=max_z_diff)

                print(f"Processing {ch_name} with tophat...")
                results_tophat = process_single_stack(
                    channel_data, apply_tophat=True, max_z_diff=max_z_diff)

                all_results[f"{ch_name}_no_tophat"] = results_no_tophat
                all_results[f"{ch_name}_tophat"] = results_tophat

                if save_results:
                    save_results_to_disk(results_no_tophat, filename, ch_name, False)
                    save_results_to_disk(results_tophat, filename, ch_name, True)

        return all_results

    else:
        print(f"Unsupported image dimensions: {len(image.shape)}")
        return None

# Main execution - just call process_directory with your parameters
def main():
    create_dirs()
    processed_files = process_directory(
        input_dir=INPUT_DIR,
        max_z_diff=DEFAULT_MAX_Z_DIFF,
        file_pattern='*.tif*',
        save_results=True
    )
    print(f"Processed {len(processed_files)} files")

if __name__ == "__main__":
    main()

Mounted at /content/drive
Found 11 files to process

Processing file 1/11: denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq001.tif
Processing file: denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq001.tif
Image shape: (3, 21, 1024, 1024), dtype: float32
Processing Cadherins without tophat...
Processing stack with shape (21, 1024, 1024)...
Normalizing stack from range [-0.046257615089416504, 3.5305984020233154] to [0, 1]
Applying regional guided EDF...
Processing Cadherins with tophat...
Processing stack with shape (21, 1024, 1024)...
Normalizing stack from range [-0.046257615089416504, 3.5305984020233154] to [0, 1]
Applying tophat background removal...
Applying regional guided EDF...
Saved /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x40/Cadherins/background/denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional.tif
Saved /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x40/Cadherins/tophat/denoised_0Pa_A1_19dec21_40x_L2RA_FlatA_seq0

In [3]:
# Import required libraries
import os
import numpy as np
from skimage import io, filters, exposure, morphology, img_as_float, img_as_uint
from scipy import ndimage
import glob
import tifffile
import matplotlib.pyplot as plt

# ======= CONFIGURATION =======
# Set paths specifically for your Google Colab environment
INPUT_DIR = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised_ordered/Static-x40'
OUTPUT_DIR = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x40'

# Default EDF parameters
DEFAULT_MAX_Z_DIFF = 1      # Maximum allowed z-difference between adjacent pixels
DEFAULT_SIGMA = 1.0         # Sigma for Laplacian operator smoothing
DEFAULT_GAUSS_DENOISE = 0.5 # Sigma for Gaussian denoising

# ======= UTILITY FUNCTIONS =======

def create_nuclei_background_dir():
    """Create the directory for Nuclei background projections"""
    nuclei_background_dir = os.path.join(OUTPUT_DIR, 'Nuclei', 'background')
    os.makedirs(nuclei_background_dir, exist_ok=True)
    print(f"Created directory: {nuclei_background_dir}")

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# ======= REGIONAL GUIDED EDF METHOD =======

def edf_regional_guided(image_stack, region_size=15, overlap=5, sigma=1.0, gauss_denoise=0.5, max_z_diff=1):
    """
    Regional guided EDF with spatial consistency
    Processes image in overlapping regions to better handle local depth variations
    """
    # Convert to float and ensure 0-1 range
    stack = img_as_float(image_stack)
    stack = normalize_image(stack)
    z_size, height, width = stack.shape

    # Calculate focus measures
    focus_measures = np.zeros((z_size, height, width))

    for z in range(z_size):
        # Apply Gaussian filter to reduce noise
        if gauss_denoise > 0:
            blurred = filters.gaussian(stack[z], sigma=gauss_denoise)
        else:
            blurred = stack[z]

        # Apply Laplacian filter (detects edges/details)
        laplacian = np.abs(filters.laplace(blurred, ksize=3))

        # Apply small Gaussian to make the decision more robust
        if sigma > 0:
            focus_measures[z] = filters.gaussian(laplacian, sigma=sigma)
        else:
            focus_measures[z] = laplacian

    # Create output arrays
    # Changed from int32 to float64 to avoid casting error
    best_z = np.zeros((height, width), dtype=np.float64)
    weights = np.zeros((height, width), dtype=np.float32)

    # Process image in overlapping regions
    for y_start in range(0, height, region_size - overlap):
        for x_start in range(0, width, region_size - overlap):
            # Define region bounds
            y_end = min(y_start + region_size, height)
            x_end = min(x_start + region_size, width)

            # Extract region
            region_focus = focus_measures[:, y_start:y_end, x_start:x_end]

            # Process region with spatial continuity
            region_best_z = edf_with_spatial_continuity_region(
                region_focus,
                max_z_diff=max_z_diff
            )

            # Create weight mask (higher weights in center, lower at edges)
            y_grid, x_grid = np.mgrid[y_start:y_end, x_start:x_end]
            y_center = (y_start + y_end) / 2
            x_center = (x_start + x_end) / 2

            # Calculate distance from center (normalized to 0-1)
            y_dist = np.abs(y_grid - y_center) / (region_size / 2)
            x_dist = np.abs(x_grid - x_center) / (region_size / 2)
            dist = np.maximum(y_dist, x_dist)
            region_weights = np.clip(1.0 - dist, 0.1, 1.0)

            # Convert region_best_z to float64 before multiplying
            region_best_z_float = region_best_z.astype(np.float64)

            # Add weighted contribution to output
            best_z[y_start:y_end, x_start:x_end] += region_best_z_float * region_weights
            weights[y_start:y_end, x_start:x_end] += region_weights

    # Normalize by weights and round to nearest integer
    best_z = np.round(best_z / np.maximum(weights, 1e-6)).astype(np.int32)

    # Ensure best_z values are within valid range
    best_z = np.clip(best_z, 0, z_size - 1)

    # Create output by taking pixels from best z-slices
    result = np.zeros((height, width), dtype=np.float32)
    for z in range(z_size):
        mask = best_z == z
        result[mask] = stack[z][mask]

    # Create a visualization of the focus map (normalized to 0-1)
    focus_map = best_z / (z_size - 1)

    return result, focus_map, best_z

def edf_with_spatial_continuity_region(focus_measures, max_z_diff=1):
    """
    Helper function for regional guided EDF
    Applies spatial continuity to a region's focus measures
    """
    z_size, height, width = focus_measures.shape

    # Find initial z-slice with maximum focus for each pixel
    initial_best_z = np.argmax(focus_measures, axis=0)

    # Create a processed mask to track pixels that have been assigned a final z-value
    processed = np.zeros((height, width), dtype=bool)

    # Create the output z-map that will be filled with spatially consistent z-values
    final_best_z = np.copy(initial_best_z)

    # Function to get valid 4-connected neighbors
    def get_neighbors(y, x):
        neighbors = []
        for ny, nx in [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]:
            if 0 <= ny < height and 0 <= nx < width:
                neighbors.append((ny, nx))
        return neighbors

    # Find seed points - we'll start from pixels with highest confidence
    confidence = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            z = initial_best_z[y, x]
            confidence[y, x] = focus_measures[z, y, x]

    # Sort pixels by confidence (highest first)
    confidence_threshold = np.percentile(confidence, 95)  # Use top 5% as seeds
    seed_indices = np.where(confidence > confidence_threshold)
    seed_points = list(zip(seed_indices[0], seed_indices[1]))

    # Sort seed points by confidence (highest first)
    seed_points.sort(key=lambda idx: confidence[idx[0], idx[1]], reverse=True)

    # Initialize frontier with seed points
    frontier = seed_points.copy()
    for y, x in frontier:
        processed[y, x] = True

    # Process each point and its neighbors using a breadth-first approach
    while frontier:
        y, x = frontier.pop(0)
        current_z = final_best_z[y, x]

        # Check neighbors
        for ny, nx in get_neighbors(y, x):
            if processed[ny, nx]:
                continue

            # Find best z within allowed range from current pixel
            z_min = max(0, current_z - max_z_diff)
            z_max = min(z_size - 1, current_z + max_z_diff)

            # Extract the relevant slice of focus measures and find best z
            z_slice = focus_measures[z_min:z_max+1, ny, nx]
            relative_best_z = np.argmax(z_slice)
            final_best_z[ny, nx] = z_min + relative_best_z

            processed[ny, nx] = True
            frontier.append((ny, nx))

    # Check if there are any unprocessed pixels left (should be rare)
    unprocessed = ~processed
    if np.any(unprocessed):
        # Assign them the value of nearest processed neighbor
        dist, indices = ndimage.distance_transform_edt(
            unprocessed, return_indices=True)

        # Assign z-values from nearest processed pixel
        for y, x in zip(*np.where(unprocessed)):
            idx_y, idx_x = indices[0, y, x], indices[1, y, x]
            final_best_z[y, x] = final_best_z[idx_y, idx_x]

    return final_best_z

# ======= SPECIFIC NUCLEI PROCESSING FUNCTIONS =======

def process_nuclei_stack(stack, max_z_diff=DEFAULT_MAX_Z_DIFF):
    """
    Process a Nuclei z-stack WITHOUT tophat filter using the regional EDF method
    """
    print(f"Processing Nuclei stack with shape {stack.shape}...")

    # Convert to float and ensure valid range (0-1)
    stack = img_as_float(stack)

    # Ensure stack values are in 0-1 range
    min_val = np.min(stack)
    max_val = np.max(stack)
    if min_val < 0 or max_val > 1:
        print(f"Normalizing stack from range [{min_val}, {max_val}] to [0, 1]")
        stack = normalize_image(stack)

    # Apply regional method WITHOUT tophat
    print("Applying regional guided EDF without tophat...")
    proj, focus_map, _ = edf_regional_guided(
        stack, max_z_diff=max_z_diff)

    return proj

def save_nuclei_projection(projection, filename):
    """Save the Nuclei projection to the background folder"""
    base_name = os.path.splitext(filename)[0]

    # Create output directory path
    output_dir = os.path.join(OUTPUT_DIR, 'Nuclei', 'background')
    os.makedirs(output_dir, exist_ok=True)

    # Create output filename
    output_filename = f"{base_name}_Nuclei_regional.tif"
    output_path = os.path.join(output_dir, output_filename)

    # Normalize to 0-1 range and convert to uint16
    projection_normalized = normalize_image(projection)
    projection_uint = img_as_uint(projection_normalized)

    # Save
    tifffile.imwrite(output_path, projection_uint)
    print(f"Saved {output_path}")

def process_all_nuclei_files(input_dir=INPUT_DIR, max_z_diff=DEFAULT_MAX_Z_DIFF, file_pattern='*.tif*'):
    """Process all files and extract Nuclei channel projections without tophat"""
    # Get all matching files
    all_files = glob.glob(os.path.join(input_dir, file_pattern), recursive=True)
    all_files.sort()

    if not all_files:
        print(f"No files matching '{file_pattern}' found in {input_dir}")
        return

    print(f"Found {len(all_files)} files to process")

    # Create output directory
    create_nuclei_background_dir()

    # Process each file
    processed_count = 0

    for file_idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"\nProcessing file {file_idx+1}/{len(all_files)}: {filename}")

        try:
            # Load image
            image = tifffile.imread(file_path)
            print(f"Image shape: {image.shape}, dtype: {image.dtype}")

            # Check if it's a multi-channel stack
            if len(image.shape) == 4:
                # Extract Nuclei channel (index 1)
                nuclei_channel = image[1]

                # Process without tophat
                projection = process_nuclei_stack(nuclei_channel, max_z_diff=max_z_diff)

                # Save result
                save_nuclei_projection(projection, filename)
                processed_count += 1

            else:
                print(f"Skipping {filename}: Not a multi-channel stack")

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            import traceback
            traceback.print_exc()

    print(f"\nSuccessfully processed {processed_count}/{len(all_files)} files")

# Main execution
if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')

    print("Processing Nuclei channel without tophat...")
    process_all_nuclei_files(
        input_dir=INPUT_DIR,
        max_z_diff=DEFAULT_MAX_Z_DIFF,
        file_pattern='*.tif*'
    )
    print("Done!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing Nuclei channel without tophat...
Found 13 files to process
Created directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x40/Nuclei/background

Processing file 1/13: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001.tif
Image shape: (3, 21, 1024, 1024), dtype: float32
Processing Nuclei stack with shape (21, 1024, 1024)...
Normalizing stack from range [-0.009803643450140953, 1.6270993947982788] to [0, 1]
Applying regional guided EDF without tophat...
Saved /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x40/Nuclei/background/denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Nuclei_regional.tif

Processing file 2/13: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq002.tif
Image shape: (3, 21, 1024, 1024), dtype: float32
Processing Nuclei stack with shape (21, 1024, 1024)...
Normalizing stack from range